# 입문자를 위한, 파이썬/R 데이터 분석

## Part(2) - Day12  : 

]

## Today's mission

- [ 분류(CLASSIFICATION) ] : 머신러닝 데이터 분석 예제 실습
1. KNN classification
2. SVM classification
3. 의사결정나무 (Decision Tree)

]

# [ 분류(CLASSIFICATION) ]

## 1. KNN classification

In [1]:
### 기본 라이브러리 불러오기
import pandas as pd
import seaborn as sns

In [2]:
'''
[Step 1] 데이터 준비 - Seaborn에서 제공하는 titanic 데이터셋 가져오기
'''
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [3]:
# 데이터 살펴보기
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
#  IPython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
'''
[Step 2] 데이터 탐색
'''
# 데이터 자료형 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [3]:
# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype='object')

In [4]:
# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
len(rdf)

714

In [8]:
rdf['embarked'].value_counts()

S    554
C    130
Q     28
Name: embarked, dtype: int64

In [5]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
most_freq

'S'

In [11]:
rdf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
rdf['embarked'].fillna(most_freq, inplace=True)

In [13]:
'''
[Step 3] 분석에 사용할 속성을 선택
'''
# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [14]:
# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)
ndf

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1
...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,Q,1,0
886,0,2,male,27.0,0,0,S,0,1
887,1,1,female,19.0,0,0,S,1,0
889,1,1,male,26.0,0,0,C,0,1


In [15]:
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)
ndf

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,Q,1,0,0,1,0
886,0,2,male,27.0,0,0,S,0,1,0,0,1
887,1,1,female,19.0,0,0,S,1,0,0,0,1
889,1,1,male,26.0,0,0,C,0,1,1,0,0


In [16]:
ndf.drop(['sex', 'embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


In [17]:
'''
[Step 4] 데이터셋 구분 - 훈련용(train data)/ 검증용(test data)
'''
# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 
       'town_C', 'town_Q', 'town_S']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

In [18]:
X.describe()

,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
count,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000,714.000000
mean,2.236695,29.699118,0.512605,0.431373,0.365546,0.634454,0.182073,0.039216,0.778711
std,0.838250,14.526497,0.929783,0.853289,0.481921,0.481921,0.386175,0.194244,0.415405
min,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2.000000,28.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,3.000000,38.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,3.000000,80.000000,5.000000,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

array([[ 0.91123237, -0.53037664,  0.52457013, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364,  0.57183099,  0.52457013, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237, -0.25482473, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       ...,
       [-1.47636364, -0.73704057, -0.55170307, ..., -0.47180795,
        -0.20203051,  0.53307848],
       [-1.47636364, -0.25482473, -0.55170307, ...,  2.11950647,
        -0.20203051, -1.87589641],
       [ 0.91123237,  0.15850313, -0.55170307, ..., -0.47180795,
         4.94974747, -1.87589641]])

In [21]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8
0,0.911232,-0.530377,0.524570,-0.505895,-0.759051,0.759051,-0.471808,-0.202031,0.533078
1,-1.476364,0.571831,0.524570,-0.505895,1.317434,-1.317434,2.119506,-0.202031,-1.875896
2,0.911232,-0.254825,-0.551703,-0.505895,1.317434,-1.317434,-0.471808,-0.202031,0.533078
3,-1.476364,0.365167,0.524570,-0.505895,1.317434,-1.317434,-0.471808,-0.202031,0.533078
4,0.911232,0.365167,-0.551703,-0.505895,-0.759051,0.759051,-0.471808,-0.202031,0.533078
...,...,...,...,...,...,...,...,...,...
709,0.911232,0.640719,-0.551703,5.357890,1.317434,-1.317434,-0.471808,4.949747,-1.875896
710,-0.282566,-0.185937,-0.551703,-0.505895,-0.759051,0.759051,-0.471808,-0.202031,0.533078
711,-1.476364,-0.737041,-0.551703,-0.505895,1.317434,-1.317434,-0.471808,-0.202031,0.533078
712,-1.476364,-0.254825,-0.551703,-0.505895,-0.759051,0.759051,2.119506,-0.202031,-1.875896


In [20]:
pd.DataFrame(X).describe()

,0,1,2,3,4,5,6,7,8
count,7.140000e+02,7.140000e+02,7.140000e+02,7.140000e+02,7.140000e+02,7.140000e+02,7.140000e+02,7.140000e+02,7.140000e+02
mean,-1.043361e-16,2.174187e-16,5.946068e-16,-1.710428e-16,2.363500e-17,3.949533e-17,-6.297484e-18,1.119553e-17,1.741526e-17
std,1.000701e+00,1.000701e+00,1.000701e+00,1.000701e+00,1.000701e+00,1.000701e+00,1.000701e+00,1.000701e+00,1.000701e+00
min,-1.476364e+00,-2.016979e+00,-5.517031e-01,-5.058951e-01,-7.590513e-01,-1.317434e+00,-4.718079e-01,-2.020305e-01,-1.875896e+00
25%,-1.476364e+00,-6.595416e-01,-5.517031e-01,-5.058951e-01,-7.590513e-01,-1.317434e+00,-4.718079e-01,-2.020305e-01,5.330785e-01
50%,-2.825656e-01,-1.170488e-01,-5.517031e-01,-5.058951e-01,-7.590513e-01,7.590513e-01,-4.718079e-01,-2.020305e-01,5.330785e-01
75%,9.112324e-01,5.718310e-01,5.245701e-01,6.668618e-01,1.317434e+00,7.590513e-01,-4.718079e-01,-2.020305e-01,5.330785e-01
max,9.112324e-01,3.465126e+00,4.829663e+00,6.530646e+00,1.317434e+00,7.590513e-01,2.119506e+00,4.949747e+00,5.330785e-01


In [22]:
# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


In [23]:
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

In [24]:
# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

KNeighborsClassifier()

In [25]:
# test data를 가지고 y_hat을 예측 (분류) 
y_hat = knn.predict(X_test)

In [26]:
print(y_hat) # 예측값
print("--------------------------------------------------------------------")
print(y_test.values) # 정답

[0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1
 0 1 1 0 1 1 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0
 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1
 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 1
 0 1 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1]
--------------------------------------------------------------------
[0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 1
 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 1 0 1
 0 1 0 0 0 0 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 1 1 0 0 1 1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 1 0
 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1
 0 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 1 0]


In [29]:
num = 0
for i,v in zip(y_hat, y_test.values):
    if i == v:
        num += 1

print(f"{num}개 정답")

177개 정답


In [48]:
print(f"정답률 : {num / len(y_hat):.2f}")

정답률 : 0.82


In [33]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, y_hat)  
print(knn_matrix)

[[111  14]
 [ 24  66]]


In [34]:
# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat)            
print(knn_report)

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       125
           1       0.82      0.73      0.78        90

    accuracy                           0.82       215
   macro avg       0.82      0.81      0.82       215
weighted avg       0.82      0.82      0.82       215



## 2. SVM classification

In [49]:
# -*- coding: utf-8 -*-
### 기본 라이브러리 불러오기
import pandas as pd
import seaborn as sns

In [50]:
'''
[Step 1] 데이터 준비/ 기본 설정
'''
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

#  IPython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)

In [51]:
'''
[Step 2] 데이터 탐색/ 전처리
'''
# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)  

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)  

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)

In [52]:
'''
[Step 3] 분석에 사용할 속성을 선택
'''
# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked'], axis=1, inplace=True)

In [53]:
ndf

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
885,0,3,39.0,0,5,1,0,0,1,0
886,0,2,27.0,0,0,0,1,0,0,1
887,1,1,19.0,0,0,1,0,0,0,1
889,1,1,26.0,0,0,0,1,1,0,0


In [54]:
'''
[Step 4] 데이터셋 구분 - 훈련용(train data)/ 검증용(test data)
'''
# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 
       'town_C', 'town_Q', 'town_S']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


In [55]:
'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

In [56]:
# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)   

SVC()

In [57]:
# test data를 가지고 y_hat을 예측 (분류) 
y_hat = svm_model.predict(X_test)

In [58]:
# 예측값
print(y_hat[0:20])
# 정답
print(y_test.values[0:20])

[0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0]
[0 0 1 0 0 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1]


In [59]:
num = 0
for i,v in zip(y_hat, y_test.values):
    if i == v:
        num += 1

print(f"정답률 : {num / len(y_hat):.2f}")

정답률 : 0.81


In [60]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat)  
svm_matrix

array([[120,   5],
       [ 35,  55]])

In [61]:
# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)            
print(svm_report)

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215



## 3. 의사결정나무 (Decision Tree)

In [71]:
# -*- coding: utf-8 -*-
### 기본 라이브러리 불러오기
import pandas as pd
import numpy as np

In [72]:
'''
[Step 1] 데이터 준비/ 기본 설정
'''
# Breast Cancer 데이터셋 가져오기 (출처: UCI ML Repository)
uci_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/\
breast-cancer-wisconsin/breast-cancer-wisconsin.data'
df = pd.read_csv(uci_path, header=None)
df

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [73]:
uci_path

'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data'

In [74]:
# 열 이름 지정
df.columns = ['id','clump','cell_size','cell_shape', 'adhesion','epithlial',
              'bare_nuclei','chromatin','normal_nucleoli', 'mitoses', 'class'] 
#  IPython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)
df

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nuclei,chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...
694,776715,3,1,1,1,3,2,1,1,1,2
695,841769,2,1,1,1,2,1,1,1,1,2
696,888820,5,10,10,3,7,3,8,10,2,4
697,897471,4,8,6,4,3,4,10,6,1,4


In [75]:
'''
[Step 2] 데이터 탐색
'''
# 데이터 살펴보기
df.head()

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nuclei,chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [76]:
# 데이터 자료형 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               699 non-null    int64 
 1   clump            699 non-null    int64 
 2   cell_size        699 non-null    int64 
 3   cell_shape       699 non-null    int64 
 4   adhesion         699 non-null    int64 
 5   epithlial        699 non-null    int64 
 6   bare_nuclei      699 non-null    object
 7   chromatin        699 non-null    int64 
 8   normal_nucleoli  699 non-null    int64 
 9   mitoses          699 non-null    int64 
 10  class            699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [77]:
# 데이터 통계 요약정보 확인
df.describe()

,id,clump,cell_size,cell_shape,adhesion,epithlial,chromatin,normal_nucleoli,mitoses,class
count,6.990000e+02,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,1.071704e+06,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,6.170957e+05,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,6.163400e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.706885e+05,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171710e+06,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238298e+06,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [78]:
# bare_nuclei 열의 자료형 변경 (문자열 ->숫자)
df['bare_nuclei'].unique()                         # bare_nuclei 열의 고유값 확인

array(['1', '10', '2', '4', '3', '9', '7', '?', '5', '8', '6'],
      dtype=object)

In [79]:
df['bare_nuclei'].replace('?', np.nan, inplace=True)      # '?'을 np.nan으로 변경
df.dropna(subset=['bare_nuclei'], axis=0, inplace=True)   # 누락데이터 행을 삭제
df['bare_nuclei'] = df['bare_nuclei'].astype('int')       # 문자열을 정수형으로 변환

In [80]:
df.describe()                                      # 데이터 통계 요약정보 확인

,id,clump,cell_size,cell_shape,adhesion,epithlial,bare_nuclei,chromatin,normal_nucleoli,mitoses,class
count,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,1.076720e+06,4.442167,3.150805,3.215227,2.830161,3.234261,3.544656,3.445095,2.869693,1.603221,2.699854
std,6.206440e+05,2.820761,3.065145,2.988581,2.864562,2.223085,3.643857,2.449697,3.052666,1.732674,0.954592
min,6.337500e+04,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,8.776170e+05,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,1.171795e+06,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,1.238705e+06,6.000000,5.000000,5.000000,4.000000,4.000000,6.000000,5.000000,4.000000,1.000000,4.000000
max,1.345435e+07,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [82]:
df['class'].unique()

array([2, 4])

In [84]:
'''
[Step 3] 데이터셋 구분 - 훈련용(train data)/ 검증용(test data)
'''
# 속성(변수) 선택
X=df[['clump','cell_size','cell_shape', 'adhesion','epithlial',
      'bare_nuclei','chromatin','normal_nucleoli', 'mitoses']]  #설명 변수 X
y=df['class']                                                   #예측 변수 Y

In [85]:
y.unique()

array([2, 4])

In [86]:
# 설명 변수 데이터를 정규화
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

In [87]:
# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

In [88]:
print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (478, 9)
test data 개수:  (205, 9)


In [89]:
'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [90]:
# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

DecisionTreeClassifier(criterion='entropy', max_depth=5)

In [95]:
# test data를 가지고 y_hat을 예측 (분류) 
y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

In [96]:
# 예측값
print(y_hat)
print("--------------------------------------")
print(y_test.values)
# 정답

[4 4 4 4 4 4 2 2 4 4 4 2 2 4 4 2 2 4 2 4 2 2 2 2 4 2 2 2 2 2 2 4 4 4 2 4 4
 4 2 2 4 2 4 2 4 2 4 2 2 2 2 2 2 2 2 2 4 4 2 4 2 2 2 4 2 2 2 2 2 2 2 2 4 2
 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2 4 2 2 2 2 2 2 2 2 2 4 2 4 4 2 2 2 2 4 2 4
 2 2 4 2 2 4 2 2 4 2 4 2 4 4 4 4 2 4 2 2 2 2 2 4 4 4 4 2 4 4 2 2 2 4 4 2 4
 2 2 2 4 2 2 2 4 4 2 2 2 2 2 4 2 4 4 2 4 4 2 2 4 4 2 2 4 2 2 2 2 4 2 2 2 4
 2 2 2 2 2 4 2 2 4 2 4 2 4 2 4 4 4 2 4 2]
--------------------------------------
[4 4 4 4 4 4 2 2 4 4 4 2 2 4 4 2 2 4 2 4 2 2 2 2 4 2 2 2 2 2 2 4 4 4 2 4 4
 4 2 2 4 2 4 2 4 2 2 2 2 2 2 2 2 2 2 2 4 4 2 4 2 4 2 4 2 2 2 2 2 2 2 2 4 2
 2 2 2 2 2 2 2 2 2 2 4 4 2 2 2 2 4 2 2 2 2 2 2 2 2 2 4 2 4 4 2 2 2 2 4 2 4
 2 2 4 2 4 4 2 2 2 2 2 2 4 4 4 4 2 4 2 2 2 2 2 4 4 4 4 2 4 4 2 2 2 4 4 2 4
 2 2 2 4 2 2 2 4 4 2 2 2 2 2 4 2 4 4 2 4 4 2 2 4 4 2 2 4 2 2 2 2 4 2 2 2 4
 2 2 2 2 2 4 2 2 2 2 4 2 4 2 4 4 4 2 4 2]


In [97]:
# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, y_hat)  
print(tree_matrix)

[[127   4]
 [  2  72]]


In [99]:
# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, y_hat)            
print(tree_report)

              precision    recall  f1-score   support

           2       0.98      0.97      0.98       131
           4       0.95      0.97      0.96        74

    accuracy                           0.97       205
   macro avg       0.97      0.97      0.97       205
weighted avg       0.97      0.97      0.97       205



## 실습)

In [114]:
'''
[Step 1] 데이터 준비 - Seaborn에서 제공하는 titanic 데이터셋 가져오기
'''
# load_dataset 함수를 사용하여 데이터프레임으로 변환
df = sns.load_dataset('titanic')

# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)

'''
[Step 3] 분석에 사용할 속성을 선택
'''
# 분석에 활용할 열(속성)을 선택 
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

# 원핫인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변환
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked'], axis=1, inplace=True)

'''
[Step 4] 데이터셋 구분 - 훈련용(train data)/ 검증용(test data)
'''
# 속성(변수) 선택
X=ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 
       'town_C', 'town_Q', 'town_S']]  #독립 변수 X
y=ndf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


In [40]:
df = sns.load_dataset('titanic')

# NaN값이 많은 deck 열을 삭제, embarked와 내용이 겹치는 embark_town 열을 삭제
rdf = df.drop(['deck', 'embark_town'], axis=1)

# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)

# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()   
rdf['embarked'].fillna(most_freq, inplace=True)


In [41]:
# 1. 전부 one-hot encoding
all_oh_rdf = rdf.copy()
all_oh_rdf['male'] = all_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
all_oh_rdf['female'] = all_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

all_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex'], axis=1, inplace=True)
all_oh_rdf['alone'] = all_oh_rdf['alone'].apply(lambda x: 1 if x == True else 0)

test = pd.get_dummies(all_oh_rdf['pclass'], prefix="pclass")
test1 = pd.get_dummies(all_oh_rdf['sibsp'], prefix="sibsp")
test2 = pd.get_dummies(all_oh_rdf['parch'], prefix="parch")
test3 = pd.get_dummies(all_oh_rdf['embarked'], prefix="embarked")

all_oh_rdf = pd.concat([all_oh_rdf, test, test1, test2, test3], axis=1)
all_oh_rdf.drop(['pclass', 'sibsp', 'parch', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=all_oh_rdf.drop(columns='survived')  #독립 변수 X
y=all_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 23)
test data 개수:  (215, 23)


In [42]:
print("전부 one-hot encondig")
print("sex, pclass, sibsp, parch, embarked")
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")       



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")         


전부 one-hot encondig
sex, pclass, sibsp, parch, embarked
[[109  16]
 [ 30  60]]

KNN 정답률 : 0.79
[[119   6]
 [ 33  57]]

SVM 정답률 : 0.82
[[118   7]
 [ 36  54]]
TREE 정답률 : 0.80


In [43]:
# 2. 기본 one-hot encoding
sex_oh_rdf = rdf.copy()
sex_oh_rdf['male'] = sex_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
sex_oh_rdf['female'] = sex_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

sex_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'alone', 'sex'], axis=1, inplace=True)

test3 = pd.get_dummies(sex_oh_rdf['embarked'], prefix="embarked")

sex_oh_rdf = pd.concat([sex_oh_rdf, test3], axis=1)
sex_oh_rdf.drop(['embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=sex_oh_rdf.drop(columns='survived')  #독립 변수 X
y=sex_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


In [44]:
print("기본 one-hot encondig")
print("sex, embarked")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


기본 one-hot encondig
sex, embarked

[[111  14]
 [ 24  66]]

KNN 정답률 : 0.82


[[120   5]
 [ 35  55]]

SVM 정답률 : 0.81


[[120   5]
 [ 35  55]]
TREE 정답률 : 0.81




In [45]:
# 3. pclass one-hot encoding
pclass_oh_rdf = rdf.copy()
pclass_oh_rdf['male'] = pclass_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
pclass_oh_rdf['female'] = pclass_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

pclass_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex', 'alone'], axis=1, inplace=True)

test = pd.get_dummies(pclass_oh_rdf['pclass'], prefix="pclass")
test3 = pd.get_dummies(pclass_oh_rdf['embarked'], prefix="embarked")

pclass_oh_rdf = pd.concat([pclass_oh_rdf, test, test3], axis=1)
pclass_oh_rdf.drop(['pclass', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=pclass_oh_rdf.drop(columns='survived')  #독립 변수 X
y=pclass_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 11)
test data 개수:  (215, 11)


In [46]:
print("pclass one-hot encondig")
print("sex, embarked, pclass")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


pclass one-hot encondig
sex, embarked, pclass

[[111  14]
 [ 25  65]]

KNN 정답률 : 0.82


[[120   5]
 [ 35  55]]

SVM 정답률 : 0.81


[[120   5]
 [ 35  55]]
TREE 정답률 : 0.81




In [47]:
# 4. pclass, sibsp one-hot encoding
pc_sib_oh_rdf = rdf.copy()
pc_sib_oh_rdf['male'] = pc_sib_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
pc_sib_oh_rdf['female'] = pc_sib_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

pc_sib_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex', 'alone'], axis=1, inplace=True)

test = pd.get_dummies(pc_sib_oh_rdf['pclass'], prefix="pclass")
test1 = pd.get_dummies(pc_sib_oh_rdf['sibsp'], prefix="sibsp")
test3 = pd.get_dummies(pc_sib_oh_rdf['embarked'], prefix="embarked")

pc_sib_oh_rdf = pd.concat([pc_sib_oh_rdf, test, test1, test3], axis=1)
pc_sib_oh_rdf.drop(['pclass', 'sibsp', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=pc_sib_oh_rdf.drop(columns='survived')  #독립 변수 X
y=pc_sib_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 16)
test data 개수:  (215, 16)


In [48]:
print("pclass, sibsp one-hot encondig")
print("sex, embarked, pclass, sibsp")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


pclass, sibsp one-hot encondig
sex, embarked, pclass, sibsp

[[110  15]
 [ 33  57]]

KNN 정답률 : 0.78


[[120   5]
 [ 36  54]]

SVM 정답률 : 0.81


[[118   7]
 [ 36  54]]
TREE 정답률 : 0.80




In [49]:
# 5. pclass parch one-hot encoding
pc_par_oh_rdf = rdf.copy()
pc_par_oh_rdf['male'] = pc_par_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
pc_par_oh_rdf['female'] = pc_par_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

pc_par_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex', 'alone'], axis=1, inplace=True)

test = pd.get_dummies(pc_par_oh_rdf['pclass'], prefix="pclass")
test2 = pd.get_dummies(pc_par_oh_rdf['parch'], prefix="parch")
test3 = pd.get_dummies(pc_par_oh_rdf['embarked'], prefix="embarked")

pc_par_oh_rdf = pd.concat([pc_par_oh_rdf, test, test2, test3], axis=1)
pc_par_oh_rdf.drop(['pclass', 'parch', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=pc_par_oh_rdf.drop(columns='survived')  #독립 변수 X
y=pc_par_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 17)
test data 개수:  (215, 17)


In [50]:
print("pclass, parch one-hot encondig")
print("sex, embarked, pclass, parch")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


pclass, parch one-hot encondig
sex, embarked, pclass, parch

[[113  12]
 [ 30  60]]

KNN 정답률 : 0.80


[[120   5]
 [ 38  52]]

SVM 정답률 : 0.80


[[120   5]
 [ 35  55]]
TREE 정답률 : 0.81




In [51]:
# 6. sibsp one-hot encoding
sib_oh_rdf = rdf.copy()
sib_oh_rdf['male'] = sib_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
sib_oh_rdf['female'] = sib_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

sib_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex', 'alone'], axis=1, inplace=True)

test1 = pd.get_dummies(sib_oh_rdf['sibsp'], prefix="sibsp")
test3 = pd.get_dummies(sib_oh_rdf['embarked'], prefix="embarked")

sib_oh_rdf = pd.concat([sib_oh_rdf, test1, test3], axis=1)
sib_oh_rdf.drop(['sibsp', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=sib_oh_rdf.drop(columns='survived')  #독립 변수 X
y=sib_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 14)
test data 개수:  (215, 14)


In [52]:
print("sibsp one-hot encondig")
print("sex, embarked, sibsp")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


sibsp one-hot encondig
sex, embarked, sibsp

[[108  17]
 [ 29  61]]

KNN 정답률 : 0.79


[[114  11]
 [ 30  60]]

SVM 정답률 : 0.81


[[118   7]
 [ 36  54]]
TREE 정답률 : 0.80




In [53]:
# 7. sibsp parch one-hot encoding
sib_par_oh_rdf = rdf.copy()
sib_par_oh_rdf['male'] = sib_par_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
sib_par_oh_rdf['female'] = sib_par_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

sib_par_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex', 'alone'], axis=1, inplace=True)

test1 = pd.get_dummies(sib_par_oh_rdf['sibsp'], prefix="sibsp")
test2 = pd.get_dummies(sib_par_oh_rdf['parch'], prefix="parch")
test3 = pd.get_dummies(sib_par_oh_rdf['embarked'], prefix="embarked")

sib_par_oh_rdf = pd.concat([sib_par_oh_rdf, test1, test2, test3], axis=1)
sib_par_oh_rdf.drop(['sibsp', 'parch', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=sib_par_oh_rdf.drop(columns='survived')  #독립 변수 X
y=sib_par_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 20)
test data 개수:  (215, 20)


In [54]:
print("sibsp parch one-hot encondig")
print("sex, embarked, sibsp, parch")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


sibsp parch one-hot encondig
sex, embarked, sibsp, parch

[[109  16]
 [ 31  59]]

KNN 정답률 : 0.78


[[114  11]
 [ 31  59]]

SVM 정답률 : 0.80


[[118   7]
 [ 36  54]]
TREE 정답률 : 0.80




In [55]:
# 8. parch one-hot encoding
parch_oh_rdf = rdf.copy()
parch_oh_rdf['male'] = parch_oh_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
parch_oh_rdf['female'] = parch_oh_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

parch_oh_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex', 'alone'], axis=1, inplace=True)

test2 = pd.get_dummies(parch_oh_rdf['parch'], prefix="parch")
test3 = pd.get_dummies(parch_oh_rdf['embarked'], prefix="embarked")

parch_oh_rdf = pd.concat([parch_oh_rdf, test1, test2, test3], axis=1)
parch_oh_rdf.drop(['parch', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=parch_oh_rdf.drop(columns='survived')  #독립 변수 X
y=parch_oh_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 21)
test data 개수:  (215, 21)


In [56]:
print("parch one-hot encondig")
print("sex, embarked, parch")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


parch one-hot encondig
sex, embarked, parch

[[108  17]
 [ 32  58]]

KNN 정답률 : 0.77


[[114  11]
 [ 31  59]]

SVM 정답률 : 0.80


[[118   7]
 [ 34  56]]
TREE 정답률 : 0.81




## 결과

### 1. 기본 one-hot 인코딩 (sex, embarked)


[[111  14]<br>
[ 24  66]]<br>
KNN 정답률 : 0.82<br>
<br>
[[120   5]<br>
[ 35  55]]<br>
SVM 정답률 : 0.81<br>
<br>
[[120   5]<br>
[ 35  55]]<br>
TREE 정답률 : 0.81<br>

### 2. pclass one-hot encoding(sex, embarked, pclass)


[[111  14]<br>
[ 25  65]]<br>
KNN 정답률 : 0.82<br>
<br>
[[120   5]<br>
[ 35  55]]<br>
SVM 정답률 : 0.81<br>
<br>
[[121   4]<br>
[ 35  55]]<br>
TREE 정답률 : 0.82<br>

### 3. pclass, sibsp one-hot encondig(sex, embarked, pclass, sibsp)


[[110  15]<br>
[ 33  57]]<br>
KNN 정답률 : 0.78<br>
<br>
[[120   5]<br>
[ 36  54]]<br>
SVM 정답률 : 0.81<br>
<br>
[[118   7]<br>
[ 36  54]]<br>
TREE 정답률 : 0.80<br>

### 4. pclass, parch one-hot encondig (sex, embarked, pclass, parch)


[[113  12]<br>
[ 30  60]]<br>
KNN 정답률 : 0.80<br>
<br>
[[120   5]<br>
[ 38  52]]<br>
SVM 정답률 : 0.80<br>
<br>
[[120   5]<br>
[ 35  55]]<br>
TREE 정답률 : 0.81<br>

### 5. sibsp one-hot encondig (sex, embarked, sibsp)


[[108  17]<br>
[ 29  61]]<br>
KNN 정답률 : 0.79<br>
<br>
[[114  11]<br>
[ 30  60]]<br>
SVM 정답률 : 0.81<br>
<br>
[[118   7]<br>
[ 36  54]]<br>
TREE 정답률 : 0.80<br>

### 6. sibsp parch one-hot encondig (sex, embarked, sibsp, parch)


[[109  16]<br>
[ 31  59]]<br>
KNN 정답률 : 0.78<br>
<br>
[[114  11]<br>
[ 31  59]]<br>
SVM 정답률 : 0.80<br>
<br>
[[118   7]<br>
[ 36  54]]<br>
TREE 정답률 : 0.80<br>

### 7. parch one-hot encondig (sex, embarked, parch)


[[108  17]<br>
[ 32  58]]<br>
KNN 정답률 : 0.77<br>
<br>
[[114  11]<br>
[ 31  59]]<br>
SVM 정답률 : 0.80<br>
<br>
[[118   7]<br>
[ 34  56]]<br>
TREE 정답률 : 0.81<br>

### 8. 전부 one-hot encondig (sex, pclass, sibsp, parch, embarked)


[[109  16]<br>
[ 30  60]]
KNN 정답률 : 0.79<br>
<br>
[[119   6]<br>
[ 33  57]]<br>
SVM 정답률 : 0.82<br>
<br>
[[118   7]<br>
[ 36  54]]<br>
TREE 정답률 : 0.80<br>










## 결론

- sibsp, parch를 one-hot 인코딩 했을때에는 결과에 크게 영향을 주지 않는다.
- 오히려 점수가 더 떨어진다.
- pclass는 one-hot 인코딩 했을때 점수가 더 높아진다.

## 의문점

- 모든 열을 0과 1의 데이터만 갖게 만들어 알고리즘을 구축하면, 오히려 구분하기 쉬워져야 하는거 아닌가?

In [12]:
test_rdf

,survived,age,sibsp,parch,alone,male,female,pclass_1,pclass_2,pclass_3,embarked_C,embarked_Q,embarked_S
0,0,22.0,1,0,0,1,0,0,0,1,0,0,1
1,1,38.0,1,0,0,0,1,1,0,0,1,0,0
2,1,26.0,0,0,1,0,1,0,0,1,0,0,1
3,1,35.0,1,0,0,0,1,1,0,0,0,0,1
4,0,35.0,0,0,1,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,39.0,0,5,0,0,1,0,0,1,0,1,0
886,0,27.0,0,0,1,1,0,0,1,0,0,0,1
887,1,19.0,0,0,1,0,1,1,0,0,0,0,1
889,1,26.0,0,0,1,1,0,1,0,0,1,0,0


In [10]:
# 1. 전부 one-hot encoding
test_rdf = rdf.copy()
test_rdf['male'] = test_rdf['sex'].apply(lambda x : 1 if x == 'male' else 0)
test_rdf['female'] = test_rdf['sex'].apply(lambda x : 1 if x == 'female' else 0)

test_rdf.drop(['class', 'who', 'adult_male', 'alive', 'fare', 'sex'], axis=1, inplace=True)
test_rdf['alone'] = test_rdf['alone'].apply(lambda x: 1 if x == True else 0)

test = pd.get_dummies(test_rdf['pclass'], prefix="pclass")
test3 = pd.get_dummies(test_rdf['embarked'], prefix="embarked")

test_rdf = pd.concat([test_rdf, test, test3], axis=1)
test_rdf.drop(['pclass', 'embarked'], axis=1, inplace=True)

# 속성(변수) 선택
X=test_rdf.drop(columns='survived')  #독립 변수 X
y=test_rdf['survived']                      #종속 변수 Y

# 설명 변수 데이터를 정규화(normalization)
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X

# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10) 

print('train data 개수: ', X_train.shape)
print('test data 개수: ', X_test.shape)

train data 개수:  (499, 12)
test data 개수:  (215, 12)


In [11]:
print("pclass one-hot encondig")
print("sex, embarked, pclass")
print()
'''
[Step 5] KNN 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 KNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k=5로 설정)
knn = KNeighborsClassifier(n_neighbors=5)

# train data를 가지고 모형 학습
knn.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
knn_y_hat = knn.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
knn_matrix = metrics.confusion_matrix(y_test, knn_y_hat)  
print(knn_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, knn_y_hat)            
num = 0
for i,v in zip(knn_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"KNN 정답률 : {num / len(knn_y_hat):.2f}")
print()
print()

'''
[Step 5] SVM 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 SVM 분류 모형 가져오기
from sklearn import svm

# 모형 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

# train data를 가지고 모형 학습
svm_model.fit(X_train, y_train)

# test data를 가지고 y_hat을 예측 (분류) 
svm_y_hat = svm_model.predict(X_test)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, svm_y_hat)  
print(svm_matrix)

print()

# 모형 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, svm_y_hat)     
num = 0
for i,v in zip(svm_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"SVM 정답률 : {num / len(svm_y_hat):.2f}")      
print()
print() 



'''
[Step 4] Decision Tree 분류 모형 - sklearn 사용
'''
# sklearn 라이브러리에서 Decision Tree 분류 모형 가져오기
from sklearn import tree

# 모형 객체 생성 (criterion='entropy' 적용)
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)

# train data를 가지고 모형 학습
tree_model.fit(X_train, y_train)   

# test data를 가지고 y_hat을 예측 (분류) 
tree_y_hat = tree_model.predict(X_test)      # 2: benign(양성), 4: malignant(악성)

# 모형 성능 평가 - Confusion Matrix 계산
from sklearn import metrics 
tree_matrix = metrics.confusion_matrix(y_test, tree_y_hat)  
print(tree_matrix)

# 모형 성능 평가 - 평가지표 계산
tree_report = metrics.classification_report(y_test, tree_y_hat)     
num = 0
for i,v in zip(tree_y_hat, y_test.values):
    if i == v:
        num += 1

print(f"TREE 정답률 : {num / len(tree_y_hat):.2f}")      
print()
print()   


pclass one-hot encondig
sex, embarked, pclass

[[111  14]
 [ 26  64]]

KNN 정답률 : 0.81


[[120   5]
 [ 35  55]]

SVM 정답률 : 0.81


[[120   5]
 [ 35  55]]
TREE 정답률 : 0.81


